# Multi-Layer Perceptrons

These NNs can be viewed as a succession of Perceptrons connected together. The added complexity of the MLP is that there can be 1 or more inner layers of neurons between the input layer and output layer.



## Architecture

In this notebook, I consider a 3 layer NN (1 input layer, 1 hidden layer, and 1 output layer):

- $L$ input nodes
- $M$ hidden nodes
- $N$ output nodes

## Activation Function

- Sigmoid Function:
$$ f(h) = \frac{1}{1+\exp(-\beta h)}$$

## Error Function

- Sum of squares of error between targets and output values:
$$ E = \frac{1}{2}\sum_{k=1}^N (y_k - t_k)^2 $$

In [1]:
import numpy as np

## Crawl before we can walk -- Revisit Logical XOR

In [2]:
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
targets = np.array([0,1,1,0]).reshape(4,1)

In [3]:
inputs

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [4]:
targets

array([[0],
       [1],
       [1],
       [0]])

In [5]:
L = 2
N = 1
M = 2 # 2 hidden nodes

### Weight Matrix

There are two sets of weights for this NN, namely: the weights connecting the input layer to the hidden layer, and the weights connecting the hidden layer to the ouput layer. I'll use two matrices to store each set of weights and store these matrices in a list.

- There are $(L+1)\times M$ weights in the first set. That is $\bf{W}_1$ is an $(L+1)\times M$ matrix, where $w_{i,j}$ gives the weight of the connection between the $i$-th input and the $j$-th node of the hidden layer.
- On the other hand, $\bf{W}_2$ is an $(M+1)\times N$ dimensional matrix. Here $w_{j,k}$ gives the weight for the connection between the $j$-th node in the hidden layer and the $k$-th output node.

In [6]:
weights = [np.zeros(((L+1)*M)).reshape((L+1),M), np.zeros((M+1)*N).reshape(M+1,N)]

In [7]:
weights

[array([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]), array([[ 0.],
        [ 0.],
        [ 0.]])]

### Initializing the weights

To avoid early saturation of the sigmoid function, the weights should be saturated to smaller values (i.e., values that are away form $\pm 1$). A common trick is to randomize the weights according to a UNIF($-1/\sqrt{n_{In}}$,$1/\sqrt{n_{In}}$), where $n_{In}$ is the number of nodes in the "input" layer to this set of weights.

In [8]:
def initializeWeights(nIn,nWeights,shape=None):
    """
    Returns an np.array of randomized weights according to the UNIF(-1/sqrt(nIn),1/sqrt(nIn)).
    """
    if shape:
        assert type(shape) == tuple
        return (-1/np.sqrt(nIn) + np.random.rand(nWeights)*(2/np.sqrt(nIn))).reshape(shape)
    else:
        return -1/np.sqrt(nIn) + np.random.rand(nWeights)*(2/np.sqrt(nIn))

In [9]:
weights1 = initializeWeights(nIn=L+1, nWeights=(L+1)*M, shape=(L+1,M))
weights1

array([[ 0.13210931,  0.40042168],
       [-0.14416024,  0.30672263],
       [ 0.18242353,  0.41802818]])

In [10]:
weights2 = initializeWeights(nIn=M+1, nWeights=(M+1)*N, shape=(L+1,N))
weights2

array([[-0.39430462],
       [-0.57286878],
       [-0.44616897]])

In [11]:
weights = [weights1,weights2]

### Forward pass

Consider the forward pass of the first input vector 

In [12]:
# Add a bias input
inputswithbias = np.concatenate((np.ones(inputs.shape[0])[:,np.newaxis],inputs), axis=1) # add bias node

In [13]:
inputswithbias

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.],
       [ 1.,  1.,  1.]])

In [39]:
thisInput = inputswithbias[0]
thisInput

array([ 1.,  0.,  0.])

In [15]:
weights1

array([[ 0.13210931,  0.40042168],
       [-0.14416024,  0.30672263],
       [ 0.18242353,  0.41802818]])

In [16]:
h1 = np.dot(thisInput,weights1)

In [17]:
h1

array([ 0.13210931,  0.40042168])

In [18]:
# activations of hidden nodes
a = 1/(1+np.exp(-h1))

In [19]:
a

array([ 0.53297938,  0.59878897])

#### Hidden nodes to Output

In [20]:
awithbias = np.ones(M+1) # should only do this the first time the algorithm runs
awithbias[1:] = a

In [21]:
awithbias

array([ 1.        ,  0.53297938,  0.59878897])

In [22]:
# compute activation of output nodse
h2 = np.dot(awithbias,weights2)

In [23]:
output = 1/(1+np.exp(-h2))

In [24]:
output

array([ 0.2755202])

### Backward pass - back propogate the error between target and output node

In [25]:
# compute error at output
deltaOut = (output - targets[0])*output*(1-output)

In [26]:
deltaOut

array([ 0.05499626])

#### Update second set of weights (output layer)

- Denote the inputs to the output nodes (i.e., the outputs of the hidden layer) as: $\mathbf{a}= [a_0,a_1,\ldots,a_M]$
- And the error at the output nodes in the row vector $\pmb{\delta}_o = [\delta_o(1),\delta_o(2),\ldots,\delta_o(N)]$

Then to update the second layer of weights, we compute the gradient matrix:
$$\pmb{\delta}_o \otimes \bf{a}^T = \begin{bmatrix}
\delta_o(1)\cdot\mathbf{a}^T,\delta_o(2)\cdot\mathbf{a}^T,\ldots,\delta_o(N)\cdot\mathbf{a}^T
\end{bmatrix}.$$

- Note that the above matrix is an $(M+1)\times N$ matrix.
- It follows that the update for the second set of weights is given by:
$$\mathbf{W}_2 \gets \mathbf{W}_2 - \eta(\pmb{\delta}_o \otimes \bf{a}^T)$$


In [27]:
eta = 0.25

In [28]:
# incrementing matrix
np.kron(awithbias.reshape(M+1,1), deltaOut)

array([[ 0.05499626],
       [ 0.02931187],
       [ 0.03293116]])

In [29]:
weights2 -= eta * np.kron(awithbias.reshape(M+1,1), deltaOut)

In [30]:
weights2

array([[-0.40805368],
       [-0.58019675],
       [-0.45440176]])

#### Update first set of weights (between input layer and hidden layer)

Using the chain rule, we can "back propogate" the error of the net to the hidden nodes.
$$\delta_h(\zeta) = \frac{\partial{E}}{\partial{h_{\zeta}}} = \sum_{k=1}^N \frac{\partial{E}}
{\partial{h_k^{out}}} \frac{\partial{h_k^{out}}}{\partial{h_{\zeta}}}$$

- This derivative asserts that the back propogation of the error is done additively. It is important to realize here that each of the hidden nodes contribute to the activations of every output node.

In [31]:
# propogate the error to the hidden layers
deltaHidden = (a*(1-a)) * np.dot(deltaOut,weights2[1:,].T)

In [32]:
deltaHidden

array([-0.00794246, -0.00600371])

- Denote the input vector as: $\mathbf{x} = [x_0,x_1,\ldots,x_L]$
- And the error at the hidden nodes in the row vector $\pmb{\delta_h} = [\delta_h(1),\delta_h(2),\ldots,\delta_h(M)]$

Then to update the first layer of weights, we compute the necessary gradient matrix:
$$\pmb{\delta}_h \otimes \mathbf{x}^T = \begin{bmatrix}
\delta_h(1)\cdot\mathbf{x}^T,\delta_h(2)\cdot\mathbf{x}^T,\ldots,\delta_h(M)\cdot\mathbf{x}^T
\end{bmatrix}$$.

- It follows that the update for the second set of weights is given by:
$$\mathbf{W}_1 \gets \mathbf{W}_1 - \eta(\pmb{\delta}_h \otimes \mathbf{x}^T)$$

In [44]:
np.kron(deltaHidden[:M],thisInput[:,np.newaxis])

array([[-0.00794246, -0.00600371],
       [-0.        , -0.        ],
       [-0.        , -0.        ]])

In [41]:
# thisInput[:,np.newaxis] # converting a row vector into a column vector

array([[ 1.],
       [ 0.],
       [ 0.]])

In [36]:
weights1 -= eta*np.kron(deltaHidden[:M],thisInput[:,np.newaxis])

In [37]:
weights1

array([[ 0.13409492,  0.40192261],
       [-0.14416024,  0.30672263],
       [ 0.18242353,  0.41802818]])

**An Important Note:** There are no connections between the bias nodes.

## Automating forward/backward phases  

In [183]:
# Initialization
np.random.seed(1)
weights1 = initializeWeights(nIn=L+1, nWeights=(L+1)*M, shape=(L+1,M))
weights2 = initializeWeights(nIn=M+1, nWeights=(M+1)*N, shape=(L+1,N))

In [184]:
(weights1,weights2)

(array([[-0.09581474,  0.25440881],
        [-0.5772182 , -0.22824668],
        [-0.40789116, -0.47072684]]), array([[-0.3622755 ],
        [-0.17833111],
        [-0.11920265]]))

In [185]:
eta = 0.25
awithbias = np.ones(M+1) # initializing the vector to store activations of hidden nodes
outputs = np.zeros(targets.shape[0])

for t in range(5000): # of iterations
    tun = np.random.permutation(inputs.shape[0])
    for i in tun: # randomize the order in which inputs are fed to the NN
        thisInput = inputswithbias[i]
        # Forward phase
        # ... computing activations of hidden nodes
        hHidden = np.dot(thisInput,weights1)
        a = 1/(1+np.exp(-hHidden))
        awithbias[1:] = a
        # ... computing activations of output nodes
        hOut = np.dot(awithbias,weights2)
        output = 1/(1+np.exp(-hOut))
        outputs[i] = output

        # Backward phase (Compute Error and Back-Propogate)
        # ... update second layer of weights
        deltaOut = (output - targets[i])*output*(1-output)
        weights2 -= eta * np.kron(deltaOut,awithbias.reshape(M+1,1))
        # ... update first layer of weights
        deltaHidden = (a*(1-a)) * np.dot(deltaOut,weights2[1:,].T)
        weights1 -= eta*np.kron(deltaHidden,thisInput[:,np.newaxis])

In [186]:
(weights1,weights2)

(array([[ 2.18818021,  6.06238454],
        [-5.71886494, -4.0987768 ],
        [-5.72736929, -4.10047476]]), array([[-3.60920341],
        [-8.16405839],
        [ 7.83638835]]))

In [187]:
outputs

array([ 0.04182017,  0.95387693,  0.9539207 ,  0.05834152])